## Setup

Let's start by importing our dependencies.

In [ ]:
# DERIVED FROM :
# https://colab.research.google.com/github/google/eng-edu/blob/master/ml/fe/exercises/intro_to_modeling.ipynb

In [ ]:
%reset -f
import numpy as np
import pandas as pd
import random as rd
import math
from matplotlib import pyplot as plt


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Set pandas output display to have one digit for decimal places and limit it to
# printing 15 rows.
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 15

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# Provide the names for the columns since the CSV file 
# with the data does not have a header row.
feature_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height', 'weight',
        'engine-type', 'num-cylinders', 'engine-size', 'fuel-system', 'bore',
        'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
        'highway-mpg', 'price']

# ORIGINAL DATA WITH '?'
car_data = pd.read_csv('https://storage.googleapis.com/mledu-datasets/cars_data.csv',
                        sep=',', names=feature_names, header=None, encoding='latin-1')

# FIND ROWS WITH BLANKS
blank_rows = list([])
blank_cols = list([])
nl_blank_rows = list([])

for label in car_data[feature_names]:
    for key,val in enumerate(car_data[label]):
        if str(val) == '?':
            if str(label) == 'num-doors':        # FILL NUM_DOORS BLANKS
                car_data.at[key,label] = 'none'
                print("Replaced num-doors blanks with 'none'")
            else:
                if str(label) == 'normalized-losses':
                    #print('Label is',label)
                    nl_blank_rows += [key]
                else:
                    print('Row', key, 'in [',label,'] has blanks.')
                    if not(key in blank_rows):    
                        blank_rows += [key]
                    if not(label in blank_cols):
                        blank_cols += [label]

blank_rows.sort()
blank_data = pd.DataFrame(car_data.iloc[blank_rows])
car_data = pd.DataFrame(car_data.drop(blank_rows))

# RANDOMIZE INDEX
blank_data.reset_index(drop=True,inplace=True)
print("Loaded and reindexed test_data.")
car_data.reset_index(drop=True,inplace=True)
car_data = car_data.reindex(np.random.permutation(np.arange(0,len(car_data))),copy=True)    
print("Loaded and reindexed car_data.")

In [ ]:
test_data

In [ ]:
norm_data = pd.DataFrame(car_data,columns=list(set(numeric_feature_names) - set(blank_cols)),copy=True)
norm_data

In [ ]:
# DEFINE VALUES

EPSILON = 0.000001  #don't divide by zero
LABEL = 'normalized-losses' #to start with
PRED = [] #collect predictions
# AVOID OVER-NORMALIZING!
ZDONE = False
LOGDONE = False
NORMDONE = False
BUCKDONE = False
SCADONE = False
CLIPPED = False

numeric_feature_names = ['symboling','normalized-losses', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-size', 'horsepower',
        'peak-rpm', 'city-mpg', 'highway-mpg', 'bore', 'stroke',
         'compression-ratio','price']
categorical_feature_names = list(set(feature_names) - set(numeric_feature_names) )#- set([LABEL]))

log_feature_names = ['wheel-base', 'horsepower', 'city-mpg', 'highway-mpg']
bucket_list = ['symboling','compression-ratio']
norm_feature_names = ['length', 'width', 'height', 'weight', 'engine-size',
                      'peak-rpm','stroke', 'bore', 'price']

#WORKING SETS TO PREPARE FOR USE IN TRAINING
x_df = pd.DataFrame( car_data[list(set(numeric_feature_names) - set(['normalized-losses']))] , dtype=float)
b_df = pd.DataFrame( blank_data[list(set(numeric_feature_names) - set(['normalized-losses']))]  )
#y_series = pd.Series( 10*(car_data[LABEL] - car_data[LABEL].min()) / 
#                    (EPSILON + car_data[LABEL].max() - car_data[LABEL].min()), 
#                    dtype=float)

# The correct count will pass these assert statements.
assert len(numeric_feature_names) == 16  #INCLUDING LABEL
assert len(categorical_feature_names) == 10

In [ ]:
#car_data[list(set(numeric_feature_names)-set(label_feature_names)-set(LABEL))]
print(LABEL)
print(blank_cols + [LABEL])
print(numeric_feature_names)
print(list(set(numeric_feature_names) - set(blank_cols + [LABEL])))#(label_feature_names) - set(LABEL)))

In [ ]:
# DEFINE SOME FUNCTIONS

#SCALE INPUT DOWN TO (0,size)
def scale_to (feature_list, size=10): #size the max scaled value
    #if not(SCADONE): 
        for feature_name in feature_list: 
            fnmax = x_df[feature_name].max()
            fnmin = x_df[feature_name].min()      
            x_df[feature_name] = size*(x_df[feature_name] - fnmin) / (fnmax - fnmin)
            print(feature_name," scaled to (",x_df[feature_name].min(),",",x_df[feature_name].max(),")") 
        #SCADONE = True
    #else:
    #    print("~~~ Data has already been clipped. ~~~")


# CLIP TO NUMBER OF STANDARD DEVIATIONS FROM MIN, MAX
def clip_by (feature_list,sigmas=1): #sigmas is how many SD's from min, max to clip
    #if not(CLIPPED):
        for feature_name in feature_list:
            fnmax = x_df[feature_name].max()
            fnmin = x_df[feature_name].min() 
            fnstd = x_df[feature_name].std() 
            for key, val in enumerate(x_df[feature_name]):
                if val > fnmax - sigmas*fnstd:              
                    x_df[feature_name][key] = fnmax - sigmas*fnstd
                if val < fnmin + sigmas*fnstd: 
                    x_df[feature_name][key] = fnmin + sigmas*fnstd
            print(feature_name, "clipped to (",fnmin + sigmas*fnstd,",",fnmax - sigmas*fnstd,") from (",fnmin,",",fnmax,")")
        #CLIPPED = True
    #else:
    #    print("~~~ Data has already been clipped. ~~~")
# END CLIP_BY

#LOG NORM
def log_norm (feature_list,LOGDONE):
    if not(LOGDONE):
        for feature_name in feature_list:
            for key, val in enumerate(x_df[feature_name]):
                if val <= 0.0:
                    print(feature_name,key," is equal to",val,"!")
                    #x_df[feature_name][key] = np.log(val+EPSILON)
                else:
                    x_df.at[key,feature_name] = np.log(val)
            print(feature_name,"log-scaled.")
        LOGDONE = True
    else:
        print("~~~ Data has already been log-normed. ~~~")

# Z NORMALIZE
def Z_norm (feature_list):
    #if not(ZDONE):        
        for feature_name in feature_list:
            fnstd = x_df[feature_name].std()
            fnmean = x_df[feature_name].mean()
            x_df[feature_name] = (x_df[feature_name] - fnmean)/(fnstd)
            print(feature_name," scaled to (",x_df[feature_name].min(),",",x_df[feature_name].max(),")")
    #   ZDONE = True
    #else:
    #    print("~~~ Data has already been Z-normed. ~~~" )

In [ ]:
# VISUALIZATION WITH PYPLOT
def scatter_plot_grid(feature_list, with_predictions=False):

    # SCATTER_PLOT PLOTS A SCATTER PLOT FOR EACH FEATURE
    def scatter_plot(axis, x_axis_fn):
        axis.set_ylabel(LABEL)
        axis.set_xlabel(x_axis_fn)
        # PLOT TRAINING SET IN GREY          
        axis.scatter(x_df[x_axis_fn], y_series, c='grey') #Y_SERIES IS SCALED SET OF LABELS
        
        if with_predictions:
            # PLOT PRICES PREDICTED FROM TEST SET DATA IN ORANGE
            axis.scatter(test_df[x_axis_fn], predictions, c='orange')
    # END OF DEFINE SCATTER_PLOT

    if with_predictions:       
        predictions = [  x['predictions'][0] for x in est.predict(predict_input_fn)   ]

    #ARRANGE SUBPLOTS AS GRID
    num_cols = 4
    num_rows = int(math.ceil(len(feature_list)/float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols)
    s = 4.5
    f.set_size_inches(num_cols*s, num_rows*s)

    # ADD EACH FEATURE PLOT TO THE GRID
    for i, fn in enumerate(feature_list):
        axis = axarr[int(i/num_cols), i%num_cols]
        scatter_plot(axis, fn)

    # CALL PYPLOT TO DISPLAY IT
    plt.show()

# END OF DEFINE SCATTER_PLOT_GRID

In [ ]:
# DISPLAY DATA SO FAR
scatter_plot_grid(numeric_feature_names)

In [ ]:
#FUNCTION PLAYGROUND

def lorm (n):
    return tf.log(n) #TF!

def loggit(fc):     #TF TENSOR --> PD SERIES
    #print("loggit")
    tx = pd.Series( fc ,copy=True,dtype=np.float32)
    #BACK TO TENSER?
    return np.log(tx) #NP!

def norm (z,mx,mn,sf=10):      #TF --> PD  --> TF
    tx = pd.Series(z,copy=True,dtype=np.float32) 
    tx = sf*((z - mn) / (mx - mn))
    #print(sf,"*((z ","-",mn,") / (",mx,"-",mn,"))")
    return tf.cast(tx,dtype=tf.float32) 

#ZCLIP TF --> ??
def zclip (fc,mx,mn,sd,sigmas=1): #sigmas is how many SD's from min, max to clip
    print("zclip")
    tx = sigclip( fc ,mx,mn,sd,sigmas) 
    return tx

#SIGCLIP TF --> ??
def sigclip(fc,mx,mn,sd,sf=1):
    print("sigclip")
    #tf.greater()
    for key, val in enumerate(tx):
        if val > mx - sf*sd:              
            tx[key] = mx - sf*sd
        if val < mn + sf*sd: 
            tx[key] = mn + sf*sd
    return tx

#ana = np.arange(10,100,step=10)
ary = pd.Series(np.arange(10,110,10,dtype=np.float32))
tary = tf.convert_to_tensor(ary)

print("series ary\n",ary)
#ala = lambda zoo: zoo*zoo
foo = lorm(tary) #, ary.max() , ary.min() )
#foo = zclip( ary, np.max(ary) , np.min(ary), np.std(ary) )
print("lorm Tary\n",foo)
#print("series from tary\n",list(tary))
#print(foo)

In [ ]:
model_feature_columns = [] #LAYERS TO FEED TO ESTIMATOR

# FOR NORMAL NUMERIC LAYERS
for fn in norm_feature_names:  
    fnmin = np.min(x_df[fn])
    fnmax = np.max(x_df[fn])
    fnstd = np.std(x_df[fn]) 
    #tx = pd.Series(10*(x_df(fn) - fnmin)/(fnmax - fnmin),dtype=np.float32,copy=True)
    tensor_column = tf.feature_column.numeric_column(fn,
                             normalizer_fn= lambda x: 10*(x - fnmin)/(fnmax - fnmin)) 
    model_feature_columns.append(tensor_column)
print("Scaled/Normalized numeric layers")

# FOR LOG NUMERIC LAYERS
for fn in log_feature_names:
    tensor_column = tf.feature_column.numeric_column(fn,
                             normalizer_fn=lambda nf: tf.log(nf))
    model_feature_columns.append(tensor_column)
print("Normalized log layers.")

# FOR BUCKET-LIST LAYERS
for fn in bucket_list:
    fnmin = min(x_df[fn])
    fnmax = max(x_df[fn])
    fnstd = x_df[fn].std()   
    tensor_column = tf.feature_column.numeric_column(fn,
                             normalizer_fn= 
                             lambda x: 10*(x - fnmin)/(fnmax - fnmin))     
    if fn == 'symboling':
        feature_bucket_step_size = 1
    if fn == 'compression-ratio':
        feature_bucket_step_size = (fnmax-fnmin)/2
    else:
        feature_bucket_step_size = (fnmax-fnmin)/12                                            
    bucket_boundaries = list(np.arange(fnmin, fnmax, 
                                       feature_bucket_step_size))
    new_column = tf.feature_column.bucketized_column(tensor_column, bucket_boundaries)
    model_feature_columns.append(new_column)
    
print("Scaled/Normalized bucket-list layers.")

print('model_feature_columns:\n')
model_feature_columns

In [ ]:
# INPUT FUNCTIONS TO SEND TO ESTIMATOR

batch_size = 12

train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True  )

eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_df,
    y=y_series,
    batch_size=batch_size,
    shuffle=False)

predict_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=test_df,
    batch_size=batch_size,
    shuffle=False)

# TRAINING

In [ ]:
# ESTIMATOR

est = tf.estimator.DNNRegressor(
    feature_columns=model_feature_columns, 
    hidden_units=[64],
    optimizer=tf.train.AdagradOptimizer(learning_rate=0.01) )

# TRAIN
num_print_statements = 20
num_training_steps = 10000
for _ in range(num_print_statements):
  est.train(train_input_fn, steps=num_training_steps // num_print_statements)
  scores = est.evaluate(eval_input_fn)
  print('SCORES:', scores,"\n\n")

In [ ]:
scatter_plot_grid(numeric_feature_names,True)